In [ ]:
!pip install pygfunction tabulate

In [ ]:
import sys
from math import pi
from pathlib import Path

import numpy as nps
import pygfunction as gt
import tabulate

In [ ]:
here = Path()
root = here / ".."
root.exists()

In [ ]:
sys.path.insert(0, str(root))

In [ ]:
from bhr.borehole import Borehole

# Comparison - PyGFunction

pygfunction example code taken from [here](https://github.com/MassimoCimmino/pygfunction/blob/9217d8d77c124239d4c1c881e34eb44c9ac6ade4/examples/custom_borehole.py)

## Single U-tube

In [ ]:
# -------------------------------------------------------------------------
# Define a single U-tube borehole
# -------------------------------------------------------------------------

# Borehole dimensions
D = 5.0  # Borehole buried depth (m)
H = 400.0  # Borehole length (m)
r_b = 0.0875  # Borehole radius (m)

# Pipe dimensions (all configurations)
epsilon = 1.0e-6  # Pipe roughness (m)

# Pipe dimensions (single U-tube and double U-tube)
r_out = 0.0211  # Pipe outer radius (m)
r_in = 0.0147  # Pipe inner radius (m)
D_s = 0.052  # Shank spacing (m)

# Ground properties
k_s = 2.0  # Ground thermal conductivity (W/m.K)

# Grout properties
k_g = 1.0  # Grout thermal conductivity (W/m.K)

# Pipe properties
k_p = 0.4  # Pipe thermal conductivity (W/m.K)

# Fluid properties
# Total fluid mass flow rate per borehole (kg/s)
m_flow_borehole = 1.0

# The fluid is propylene-glycol (20 %) at 20 degC
fluid = gt.media.Fluid("MPG", 20.0)
cp_f = fluid.cp  # Fluid specific isobaric heat capacity (J/kg.K)
rho_f = fluid.rho  # Fluid density (kg/m3)
mu_f = fluid.mu  # Fluid dynamic viscosity (kg/m.s)
k_f = fluid.k  # Fluid thermal conductivity (W/m.K)

# -------------------------------------------------------------------------
# Initialize borehole model
# -------------------------------------------------------------------------

borehole = gt.boreholes.Borehole(H, D, r_b, x=0.0, y=0.0)

# -------------------------------------------------------------------------
# Define a single U-tube borehole
# -------------------------------------------------------------------------

# Pipe positions
# Single U-tube [(x_in, y_in), (x_out, y_out)]
pos_single = [(-D_s, 0.0), (D_s, 0.0)]

# Pipe thermal resistance
R_p = gt.pipes.conduction_thermal_resistance_circular_pipe(r_in, r_out, k_p)

# Fluid to inner pipe wall thermal resistance
m_flow_pipe = m_flow_borehole
h_f = gt.pipes.convective_heat_transfer_coefficient_circular_pipe(m_flow_pipe, r_in, mu_f, rho_f, k_f, cp_f, epsilon)
R_f = 1.0 / (h_f * 2 * pi * r_in)

# Single U-tube GHE in borehole
SingleUTube = gt.pipes.SingleUTube(pos_single, r_in, r_out, borehole, k_s, k_g, R_f + R_p)

# Evaluate and print the effective borehole thermal resistance
R_b = SingleUTube.effective_borehole_thermal_resistance(m_flow_borehole, fluid.cp)

In [ ]:
single_bhr = Borehole()
single_bhr.init_single_u_borehole(
    borehole_diameter=r_b * 2,
    pipe_outer_diameter=r_out * 2,
    pipe_dimension_ratio=(2 * r_out) / (r_out - r_in),
    length=H,
    shank_space=D_s,
    pipe_conductivity=k_p,
    grout_conductivity=k_g,
    soil_conductivity=k_s,
    fluid_type="PROPYLENEGLYCOL",
    fluid_concentration=0.2,
)

In [ ]:
temp = 20

single_data = [
    ["", "pygfunction", "BHResist"],
    ["Borehole Dia.", r_b * 2, single_bhr._bh.borehole_diameter],
    ["Pipe Outer Dia.", r_out * 2, single_bhr._bh.pipe_outer_diameter],
    ["Pipe Inner Dia.", r_in * 2, single_bhr._bh.pipe_inner_diameter],
    ["Shank Spacing", D_s, single_bhr._bh.shank_space],
    ["Rp", f"{R_p:0.5f}", f"{single_bhr._bh.calc_pipe_cond_resist():0.5f}"],
    ["Rf", f"{R_f:0.5f}", f"{single_bhr._bh.calc_pipe_internal_conv_resist(m_flow_borehole, temp):0.5f}"],
    ["Rb", f"{R_b:0.5f}", f"{single_bhr.calc_bh_resist(m_flow_borehole, temp):0.5f}"],
]

table_single = tabulate.tabulate(single_data, tablefmt="html")
table_single

In [ ]:
print(f"Percent Error: {(single_bhr.calc_bh_resist(m_flow_borehole, temp) - R_b) / R_b * 100:0.3f}%")

## Double U-tube 

In [ ]:
# -------------------------------------------------------------------------
# Define a double U-tube borehole
# -------------------------------------------------------------------------

# Pipe positions
# Double U-tube [(x_in1, y_in1), (x_in2, y_in2),
#                (x_out1, y_out1), (x_out2, y_out2)]
# Note: in series configuration, fluid enters pipe (in,1), exits (out,1),
# then enters (in,2) and finally exits (out,2)
# (if you view visualize_pipe, series is 1->3->2->4)
pos_double = [(-D_s, 0.0), (0.0, -D_s), (D_s, 0.0), (0.0, D_s)]

# Pipe thermal resistance
R_p = gt.pipes.conduction_thermal_resistance_circular_pipe(r_in, r_out, k_p)

# Fluid to inner pipe wall thermal resistance

# Double U-tube in parallel
m_flow_pipe_parallel = m_flow_borehole / 2
h_f_parallel = gt.pipes.convective_heat_transfer_coefficient_circular_pipe(
    m_flow_pipe_parallel, r_in, mu_f, rho_f, k_f, cp_f, epsilon
)
R_f_parallel = 1.0 / (h_f_parallel * 2 * pi * r_in)

# Double U-tube in parallel
DoubleUTube_parallel = gt.pipes.MultipleUTube(
    pos_double, r_in, r_out, borehole, k_s, k_g, R_p + R_f_parallel, 2, config="parallel"
)

# Evaluate and print the effective borehole thermal resistance
R_b_parallel = DoubleUTube_parallel.effective_borehole_thermal_resistance(m_flow_borehole, fluid.cp)

In [ ]:
double_bhr = Borehole()
double_bhr.init_double_u_borehole(
    borehole_diameter=r_b * 2,
    pipe_outer_diameter=r_out * 2,
    pipe_dimension_ratio=(2 * r_out) / (r_out - r_in),
    length=H,
    shank_space=D_s,
    pipe_conductivity=k_p,
    pipe_inlet_arrangement="DIAGONAL",
    grout_conductivity=k_g,
    soil_conductivity=k_s,
    fluid_type="PROPYLENEGLYCOL",
    fluid_concentration=0.2,
    boundary_condition="UNIFORM_BOREHOLE_WALL_TEMP",
)

In [ ]:
double_data = [
    ["", "pygfunction", "BHResist"],
    ["Borehole Dia.", r_b * 2, double_bhr._bh.borehole_diameter],
    ["Pipe Outer Dia.", r_out * 2, double_bhr._bh.pipe_outer_diameter],
    ["Pipe Inner Dia.", r_in * 2, double_bhr._bh.pipe_inner_diameter],
    ["Shank Spacing", D_s, double_bhr._bh.shank_space],
    ["Rp", f"{R_p:0.5f}", f"{double_bhr._bh.calc_pipe_cond_resist():0.5f}"],
    ["Rf", f"{R_f_parallel:0.5f}", f"{double_bhr._bh.calc_pipe_internal_conv_resist(m_flow_pipe_parallel, temp):0.5f}"],
    ["Rb", f"{R_b_parallel:0.5f}", f"{double_bhr.calc_bh_resist(m_flow_borehole, temp):0.5f}"],
]

double_single = tabulate.tabulate(double_data, tablefmt="html")
double_single

In [ ]:
print(f"Percent Error: {(double_bhr.calc_bh_resist(m_flow_borehole, temp) - R_b_parallel) / R_b_parallel * 100:0.3f}%")

## Coaxial

In [ ]:
# -------------------------------------------------------------------------
# Define a coaxial borehole
# -------------------------------------------------------------------------

# Pipe dimensions (coaxial)
r_in_in = 0.0221  # Inside pipe inner radius (m)
r_in_out = 0.025  # Inside pipe outer radius (m)
r_out_in = 0.0487  # Outer pipe inside radius (m)
r_out_out = 0.055  # Outer pipe outside radius (m)
# Vectors of inner and outer pipe radii
# Note : The dimensions of the inlet pipe are the first elements of
#        the vectors. In this example, the inlet pipe is the inside pipe.
r_inner = np.array([r_in_in, r_out_in])  # Inner pipe radii (m)
r_outer = np.array([r_in_out, r_out_out])  # Outer pip radii (m)

# Pipe positions
# Coaxial pipe (x, y)
pos = (0.0, 0.0)

# Pipe thermal resistance
# (the two pipes have the same thermal conductivity, k_p)
# Inner pipe
R_p_in = gt.pipes.conduction_thermal_resistance_circular_pipe(r_in_in, r_in_out, k_p)
# Outer pipe
R_p_out = gt.pipes.conduction_thermal_resistance_circular_pipe(r_out_in, r_out_out, k_p)

# Fluid-to-fluid thermal resistance
# Inner pipe
h_f_in = gt.pipes.convective_heat_transfer_coefficient_circular_pipe(
    m_flow_borehole, r_in_in, mu_f, rho_f, k_f, cp_f, epsilon
)
R_f_in = 1.0 / (h_f_in * 2 * pi * r_in_in)
# Outer pipe
h_f_a_in, h_f_a_out = gt.pipes.convective_heat_transfer_coefficient_concentric_annulus(
    m_flow_borehole, r_in_out, r_out_in, mu_f, rho_f, k_f, cp_f, epsilon
)
R_f_out_in = 1.0 / (h_f_a_in * 2 * pi * r_in_out)
R_ff = R_f_in + R_p_in + R_f_out_in

# Coaxial GHE in borehole
R_f_out_out = 1.0 / (h_f_a_out * 2 * pi * r_out_in)
R_fp = R_p_out + R_f_out_out
Coaxial = gt.pipes.Coaxial(pos, r_inner, r_outer, borehole, k_s, k_g, R_ff, R_fp, J=2)

# Evaluate and print the effective borehole thermal resistance
R_b_coax = Coaxial.effective_borehole_thermal_resistance(m_flow_borehole, fluid.cp)

In [ ]:
coaxial_bhr = Borehole()
coaxial_bhr.init_coaxial_borehole(
    borehole_diameter=r_b * 2,
    outer_pipe_outer_diameter=r_out_out * 2,
    outer_pipe_dimension_ratio=(2 * r_out_out) / (r_out_out - r_out_in),
    outer_pipe_conductivity=k_p,
    inner_pipe_outer_diameter=r_in_out * 2,
    inner_pipe_dimension_ratio=(2 * r_in_out) / (r_in_out - r_in_in),
    inner_pipe_conductivity=k_p,
    length=H,
    grout_conductivity=k_g,
    soil_conductivity=k_s,
    fluid_type="PROPYLENEGLYCOL",
    fluid_concentration=0.2,
)

In [ ]:
coaxial_data = [
    ["", "pygfunction", "BHResist"],
    ["Rb", f"{R_b_coax:0.5f}", f"{coaxial_bhr.calc_bh_resist(m_flow_borehole, temp):0.5f}"],
]

coaxial_data = tabulate.tabulate(coaxial_data, tablefmt="html")
coaxial_data

In [ ]:
print(f"Percent Error: {(coaxial_bhr.calc_bh_resist(m_flow_borehole, temp) - R_b_coax) / R_b_coax * 100:0.3f}%")